In [1]:
from fetchYahooStockData import metadata_price_action, extended_stock_stats, basic_stats, current_sp500_symbols, parse_value
import numpy as np
import pandas as pd
from scipy import stats


In [2]:
symbol="V"
timeInterval = "1d"
timeRange = "1y"

In [3]:
sp500symbols = np.array(current_sp500_symbols())
sp500symbols

array(['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP',
       'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN',
       'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE',
       'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME',
       'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AAPL',
       'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK',
       'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX',
       'BDX', 'BRK.B', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA', 'BKNG', 'BWA',
       'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW', 'COG', 'CDNS',
       'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT',
       'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN', 'CF', 'CRL',
       'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS',
       'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH',
       'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'C

In [4]:
my_columns = ["Ticker", "Stock Price","4 Week Change", "4 Week Percentile", "12 Week Change", "12 Week Percentile", "26 Week Change", "26 Week Percentile", "52 Week Change", "52 Week Percentile", "HQM Score"]
dataframe = pd.DataFrame(columns=my_columns)
for sym in sp500symbols[1:10]:
    if(sym.find(".") > -1):
        sym = sym.replace(".", "-")
    basic = basic_stats(sym)
    complexStats = extended_stock_stats(sym)
    previousClose = parse_value(basic["Previous Close"])
    metadataAndPrice = metadata_price_action(sym, "1d", "1y")
    if(len(metadataAndPrice["price"]) >= 250):
        dataLength = len(metadataAndPrice["price"])
        fourWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[dataLength-19]["close"])/metadataAndPrice["price"].iloc[dataLength-19]["close"]
        twelveWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[dataLength-59]["close"])/metadataAndPrice["price"].iloc[dataLength-59]["close"]
        twentySixWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[dataLength-129]["close"])/metadataAndPrice["price"].iloc[dataLength-129]["close"]
        fiftyTwoWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[0]["close"])/metadataAndPrice["price"].iloc[0]["close"]
        dataframe = dataframe.append(pd.Series([sym,previousClose,fourWeekChange, "N/A", twelveWeekChange, "N/A", twentySixWeekChange, "N/A", fiftyTwoWeekChange, "N/A", "N/A"], index=my_columns), ignore_index=True)
        print(sym, previousClose, fourWeekChange, twelveWeekChange, twentySixWeekChange, fiftyTwoWeekChange)
# dataframe.sort_values("52 Week Change", ascending=False, inplace=True)
dataframe.reset_index(inplace=True)
dataframe.to_csv("lastBetterMomentumDataframe_bak.csv")
# dataframe = dataframe[:50]
dataframe

https://query1.finance.yahoo.com/v8/finance/chart/ABT?symbol=ABT&region=US&lang=en-US&includePrePost=false&interval=1d&useYfid=true&range=1y&corsDomain=finance.yahoo.com&.tsrc=finance
Forbidden


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
dataframe.to_csv("momentum_almost.csv")

In [16]:
time_periods = [
                '52 Week',
                '26 Week',
                '12 Week',
                '4 Week'
                ]

for row in dataframe.index:
    for time_period in time_periods:
        dataframe.loc[row, f'{time_period} Percentile'] = stats.percentileofscore(dataframe[f'{time_period} Change'], dataframe.loc[row, f'{time_period} Change'])/100

from statistics import mean

for row in dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(dataframe.loc[row, f'{time_period} Percentile'])
    dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
sorted_dataframe = dataframe.sort_values(by = 'HQM Score', ascending = False)
sorted_dataframe.to_csv("momentum_last.csv")
sorted_dataframe

,index,Ticker,Stock Price,4 Week Change,4 Week Percentile,12 Week Change,12 Week Percentile,26 Week Change,26 Week Percentile,52 Week Change,52 Week Percentile,HQM Score
446,446,TWTR,74.5,0.463350,0.998004,0.631590,0.988024,0.903186,0.968064,1.334444,0.978044,0.983034
469,469,VIAC,62.5,0.232371,0.932136,0.773165,0.994012,1.333213,0.996008,1.699456,0.992016,0.978543
143,143,DISCA,51.3,0.259022,0.9501,0.869228,1,1.324858,0.992016,1.149574,0.966068,0.977046
429,429,TPR,42.4,0.323077,0.988024,0.479635,0.984032,1.748858,1,0.836165,0.93014,0.975549
144,144,DISCK,43.4,0.285714,0.964072,0.782884,0.996008,1.176015,0.99002,0.891551,0.94012,0.972555
...,...,...,...,...,...,...,...,...,...,...,...,...
120,120,ED,67.1,-0.072085,0.0978044,-0.147735,0.0199601,-0.116658,0.0658683,-0.210843,0.0279441,0.0528942
481,481,WEC,81.7,-0.089328,0.0578842,-0.157895,0.00998004,-0.125759,0.0558882,-0.156132,0.0678643,0.0479042
470,470,VTRS,15.0,-0.148997,0.011976,-0.133100,0.0339321,-0.090631,0.0898204,-0.205032,0.0339321,0.0424152
364,364,PRGO,40.7,-0.060521,0.131737,-0.158465,0.00798403,-0.228888,0.00399202,-0.221150,0.0199601,0.0409182


In [16]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

{'currency': 'USD',
 'symbol': 'V',
 'exchangeName': 'NYQ',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 1205933400,
 'regularMarketTime': 1614373202,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 212.39,
 'chartPreviousClose': 187.21,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614330000,
   'end': 1614349800,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614349800,
   'end': 1614373200,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614373200,
   'end': 1614387600,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '1y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [17]:
price = metadataAndPrice["price"]
price

,timestamp,high,volume,close,open,low
0,1582813800,186.339996,17536600,180.009995,182.119995,180.000000
1,1582900200,182.330002,22674700,181.759995,173.789993,172.979996
2,1583159400,192.440002,17434500,192.330002,186.320007,183.100006
3,1583245800,194.039993,15226400,185.729996,193.199997,184.500000
4,1583332200,194.490005,10789100,194.289993,190.360001,186.169998
...,...,...,...,...,...,...
248,1614004200,210.649994,8332700,208.320007,203.550003,203.110001
249,1614090600,213.910004,10568500,212.110001,207.399994,206.550003
250,1614177000,220.529999,12005100,219.429993,212.360001,211.289993
251,1614263400,220.449997,11400200,213.750000,219.289993,212.580002


In [18]:
basicStats = basic_stats(symbol)
basicStats

STATS <Response [200]>


{'Previous Close': '213.75',
 'Open': '214.05',
 'Bid': '212.01 x 900',
 'Ask': '212.74 x 3100',
 "Day's Range": '210.74 - 215.21',
 '52 Week Range': '133.93 - 220.53',
 'Volume': '11,730,004',
 'Avg. Volume': '9,401,263',
 'Market Cap': '468.288B',
 'Beta (5Y Monthly)': '0.98',
 'PE Ratio (TTM)': '43.85',
 'EPS (TTM)': '4.84',
 'Earnings Date': 'Apr 28, 2021 - May 03, 2021',
 'Forward Dividend & Yield': '1.28 (0.60%)',
 'Ex-Dividend Date': 'Feb 11, 2021'}

In [19]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['483.81B',
  '482.33B',
  '438.37B',
  '424.23B',
  '354.81B',
  '416.79B'],
 'Enterprise Value 3': ['490.51B',
  '491.45B',
  '447.91B',
  '435.44B',
  '364.29B',
  '426.91B'],
 'Trailing P/E ': ['45.24', '44.73', '38.02', '34.74', '29.40', '35.32'],
 'Forward P/E 1': ['40.65', '39.84', '34.25', '32.15', '28.74', '30.12'],
 'PEG Ratio (5 yr expected) 1': ['1.88',
  '3.69',
  '2.35',
  '2.37',
  '2.01',
  '2.06'],
 'Price/Sales (ttm)': ['22.61', '22.26', '19.52', '18.17', '15.47', '18.58'],
 'Price/Book (mrq)': ['13.80', '15.50', '14.06', '14.12', '11.90', '14.26'],
 'Enterprise Value/Revenue 3': ['22.84',
  '86.42',
  '87.81',
  '90.02',
  '62.23',
  '70.52'],
 'Enterprise Value/EBITDA 6': ['32.95',
  '120.42',
  '131.89',
  '133.12',
  '88.01',
  '101.69'],
 'Beta (5Y Monthly) ': '0.98',
 '52-Week Change 3': '11.14%',
 'S&P500 52-Week Change 3': '16.75%',
 '52 Week High 3': '220.53',
 '52 Week Low 3': '133.93',
 '50-Day Moving Average 3': '206.01',
 '200-